<a href="https://colab.research.google.com/github/rposhala/Recommender-System-on-MovieLens-dataset/blob/main/Item_based_Collaborative_Recommender_System_using_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### KMeans clusteting model to build item-based collaborative Recommender System.

In [31]:
import os
import numpy as np
import pandas as pd
import pickle
from sklearn.cluster import KMeans
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.metrics import davies_bouldin_score
from sklearn.metrics import calinski_harabasz_score
import time
start = time.time()

#### Loading param

In [32]:
score_param = pd.read_csv("../data/movie_reco_scores.csv", sep = ',')

,retrain,db_score_reco_user,ch_score_reco_user,db_score_reco_movie,ch_score_reco_movie
0,0,0,0,0,0


## Loading refined_dataset

In [2]:
refined_dataset = pd.read_csv("../data/refined_dataset.csv")

#### Working test a small dataset

In [3]:
refined_dataset = refined_dataset.loc[0:1000000,:]

In [4]:
refined_dataset = refined_dataset.drop('Unnamed: 0', axis='columns')

### Recommender System by User
#### Inputs = userId
#### output = 5 movies

In [5]:
dataset_user = refined_dataset.loc[:, ['movieId', 'rating']] 

In [6]:
kmeans_user = KMeans(25)
kmeans_user.fit(dataset_user)

# Get the cluster labels
labels = kmeans_user.labels_

# Calculate Davies-Bouldin Index
db_score_reco_user = davies_bouldin_score(dataset_user, labels)

# Calculate silhouette score
#si_score = silhouette_score(dataset_user, labels)
#print("Silhouette score:", si_score)

ch_score_reco_user = calinski_harabasz_score(dataset_user, labels)


C:\Users\rafae\Desktop\MLOps\PF\venv\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Davies-Bouldin score: 0.4551964714730443
Calinski Harabasz score: 8366490.908058339


In [7]:
identified_users= kmeans_user.fit_predict(dataset_user)
identified_users = list(identified_users)

C:\Users\rafae\Desktop\MLOps\PF\venv\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [8]:
refined_dataset['loc_clusters_users'] = identified_users

### Recommender System by Movie
#### Inputs = movieId
#### Output = 5 movies

In [9]:
dataset_movie = refined_dataset.loc[:, ['userId', 'rating']] 

In [10]:
kmeans_movie = KMeans(25)
kmeans_movie.fit(dataset_user)

# Get the cluster labels
labels = kmeans_movie.labels_

# Calculate Davies-Bouldin Index
db_score_reco_movie = davies_bouldin_score(dataset_movie, labels)

# Calculate silhouette score
#si_score = silhouette_score(dataset_movie, labels)
#print("Silhouette score:", si_score)

ch_score_reco_movie = calinski_harabasz_score(dataset_movie, labels)

C:\Users\rafae\Desktop\MLOps\PF\venv\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Davies-Bouldin score: 1482.8706089044347
Calinski Harabasz score: 18.631486458567313


In [11]:
identified_movies = kmeans_movie.fit_predict(dataset_movie)
identified_movies = list(identified_movies)

C:\Users\rafae\Desktop\MLOps\PF\venv\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [12]:
refined_dataset['loc_clusters_movies'] = identified_movies

### Export the model 

In [13]:
with open("../models/kmeans_movie.pkl", "wb") as f:
    pickle.dump(kmeans_movie, f)

In [14]:
with open("../models/kmeans_user.pkl", "wb") as f:
    pickle.dump(kmeans_user, f)

### Export the final_db ti csv and pickle

In [15]:
final_db = refined_dataset

In [16]:
final_db.to_pickle("../data/final_db.pkl")

In [17]:
final_db.to_csv('../data/final_db.csv', sep=',', encoding='utf-8', index=False)

### Adding param to score csv

In [37]:
retrain = score_param['retrain'].max() + 1
new_row = {'retrain': retrain, 'db_score_reco_user': db_score_reco_user, 'ch_score_reco_user': ch_score_reco_user, 'db_score_reco_movie': db_score_reco_movie, 'ch_score_reco_movie': ch_score_reco_movie}
#print(new_row)
score_param = score_param.append(new_row, ignore_index=True)
score_param.to_csv('../data/movie_reco_scores.csv', sep=',', encoding='utf-8', index=False)

C:\Users\rafae\AppData\Local\Temp\ipykernel_7020\3250484708.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  score_param = score_param.append(new_row, ignore_index=True)


In [38]:
end = time.time()

print('Retrain done in :',end - start, 'seconds')

Retrain done in : 167.28042316436768 seconds
